In [1]:
from alexnet import AlexNet
import tensorflow as tf
from sklearn.utils import shuffle
from _preprocess import *
import time

In [2]:
features = tf.placeholder(tf.float32, (None, 227, 227, 3))
labels = tf.placeholder(tf.int64, None)
classes = 128

In [3]:
from sklearn.model_selection import train_test_split
X, y = preprocess_train(False)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=22)

Finished processing 2000
Finished processing 4000
Finished processing 6000
Finished processing 8000
Finished processing 10000
Finished processing 12000
Finished processing 14000
Finished processing 16000
Finished processing 18000
Finished processing 20000
Finished processing 22000
Finished processing 24000
Finished processing 26000
Finished processing 28000
Finished processing 30000
Finished processing 32000
Finished processing 34000
Finished processing 36000
Finished processing 38000
Finished processing 40000
Finished processing 42000
Finished processing 44000
Finished processing 46000
Finished processing 48000
Finished processing 50000
Finished processing 52000
Finished processing 54000
Finished processing 56000
Finished processing 58000
Finished processing 60000
Finished processing 62000
Finished processing 64000
Finished processing 66000
Finished processing 68000
Finished processing 70000
Finished processing 72000
Finished processing 74000
Finished processing 76000
Finished process

MemoryError: 

In [12]:
X_train = X[:160000]
X_val = X[160000:]
y_train = y[:160000]
y_val = y[160000:]

In [13]:
y_train = y_train - 1
y_val = y_val - 1

In [6]:
np.save('X.npy', X)

In [7]:
np.save('y.npy', y)

In [14]:
fc7 = AlexNet(features, feature_extract=True)
fc7 = tf.stop_gradient(fc7)
shape = (fc7.get_shape().as_list()[-1], classes)
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(classes))
logits = tf.nn.xw_plus_b(fc7, fc8W, fc8b)

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = labels, logits = logits)
loss_op = tf.reduce_mean(cross_entropy)
opt = tf.train.AdamOptimizer()
train_op = opt.minimize(loss_op, var_list=[fc8W, fc8b])
init_op = tf.global_variables_initializer()

preds = tf.argmax(logits, 1)
accuracy_op = tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))

In [15]:
epochs = 10
batch_size = 128
def eval_on_data(X, y, sess):
    total_acc = 0
    total_loss = 0
    for offset in range(0, X.shape[0], batch_size):
        end = offset + batch_size
        X_batch = X[offset:end]
        y_batch = y[offset:end]

        loss, acc = sess.run([loss_op, accuracy_op], feed_dict={features: X_batch, labels: y_batch})
        total_loss += (loss * X_batch.shape[0])
        total_acc += (acc * X_batch.shape[0])

    return total_loss/X.shape[0], total_acc/X.shape[0]

In [16]:
len(y_train)

160000

In [ ]:
with tf.Session() as sess:
    sess.run(init_op)

    for i in range(epochs):
        # training
#         X_train, y_train = shuffle(X_train, y_train)
        t0 = time.time()
        for offset in range(0, X_train.shape[0], batch_size):
            end = offset + batch_size
            sess.run(train_op, feed_dict={features: X_train[offset:end], labels: y_train[offset:end]})

        val_loss, val_acc = eval_on_data(X_val, y_val, sess)
        print("Epoch", i+1)
        print("Time: %.3f seconds" % (time.time() - t0))
        print("Validation Loss =", val_loss)
        print("Validation Accuracy =", val_acc)
        print("")

Epoch 1
Time: 547.099 seconds
Validation Loss = 1.39072235032
Validation Accuracy = 0.650908443216

Epoch 2
Time: 392.591 seconds
Validation Loss = 1.42471905827
Validation Accuracy = 0.657784111189

